# Исследование надёжности заёмщиков – анализ банковских данных

## Описание проекта

**Задачи проекта:** На основе статистики о платёжеспособности клиентов исследовать влияние семейного положения, количества детей, уровня дохода и целей кредитования на факт возврата кредита в срок.

**Данные для анализа:** Статистика о платёжеспособности клиентов.

**Этапы проекта:** 
- [загрузка данных](#uploading)
- [предобработка данных](#preprocessing)
>- [обработка пропусков](#nan)
>- [обработка дубликатов](#duplicates)
>- [обработка прочих ошибок данных](#mistakes)
>- [обработка полных дубликатов](#full_duplicates)
- [исследовательский анализ данных](#correlation)
>- [анализ зависимости между семейным положением и возвратом кредита в срок](#family_status_cor)
>- [анализ зависимости между количеством детей и возвратом кредита в срок](#children_cor)
>- [анализ зависимости между уровнем дохода и возвратом кредита в срок](#income_cor)
>- [анализ влияния цели кредитования на возврат кредита в срок](#purpose_cor)
- [Общий вывод и рекомендации](#conclusion)

<a id="uploading"></a> 
## Открытие файла с данными и изучение общей информации

In [1]:
# импортируем необходимые библиотеки
import pandas as pd

In [2]:
# загружаем датасет, выводим первые 5 строк и читаем info по датасету
df = pd.read_csv('data_borrowers.csv')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В таблице 21 525 строк и 12 столбцов, в т.ч.:
- тип данных float64 – 2 столбца:
> days_employed / общий трудовой стаж в днях,   
> total_income / ежемесячный доход,
- тип данных int64 – 5 столбцов:
> children / кол-во детей в семье,    
> dob_years / возраст клиента в годах,   
> education_id / идентификатор уровня образования,    
> family_status_id / идентификатор семейного положения,    
> debt / имел ли задолженность по возврату кредитов,   
- тип данных object – 5 столбцов:
> education / уровень образования клиента,   
> family_status / семейное положение,   
> gender / пол клиента,   
> income_type / тип занятости,   
> purpose / цель получения кредита.

Визуальный анализ данных выявил следующие ошибки данных:
- В столбцах 'days_employed' и 'total_income' есть пропущенные значения.

- Значения в столбце 'days_employed' представлены в десятичном формате и во многих случаях являются отрицательным. А в тех случаях когда значения положительные, порядок чисел существенно больше, чем у отрицательных. Это позволяет предположить, что положительные значения являются не днями, а какими-то более мелкими единицах измерения.

- Значения в столбце 'education' написаны с использованием разного регистра.

<a id="preprocessing"></a> 
## Предобработка данных

<a id="nan"></a> 
### Пропуски

Значения отсутствуют в столбцах 'days_employed' и 'total_income'. Посчитаем кол-во пропусков в каждом столбце.

In [3]:
print('Кол-во пропусков в days_employed:', df['days_employed'].isnull().sum())
print('Кол-во пропусков в total_income:', df['total_income'].isnull().sum())

Кол-во пропусков в days_employed: 2174
Кол-во пропусков в total_income: 2174


Количество отсутствующих значений одинаково для обоих столбцов – 2174. Можно предположить, что значения отсутствуют симметрично.

Проверим, вызвав список строк с пропусками значений.

In [4]:
print('Всего строк с пропусками:', len(df[df['days_employed'].isnull()]))
df[df['days_employed'].isnull()].head()

Всего строк с пропусками: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Предположение подтвердилось: где нет данных о стаже, там нет данных о доходе.

#### Трудовой стаж в днях

Кроме пропусков в столбце 'days_employed' есть еще ряд ошибок:
- данные имеют формат float, хотя из логики данных должен быть int, потому что это кол-во дней,
- часть данных является отрицательными значениями,
- положительные значения имеют порядок чисел, который существенно больше возможного для стажа в днях и не поддается интерпретации.

Поскольку данные о стаже вызывают много вопросов к их качеству и не входят в перечень параметров, которые необходимо проанализировать с точки зрения влияния на возврат кредита в срок, то можно оставить эти ошибки без обработки, заполнив только пропуски на условное значение – маркер – -1.

После операции замены убедимся, что в столбце 'days_employed' больше нет пропусков.

In [5]:
df['days_employed'] = df['days_employed'].fillna(-1)
print('Кол-во пропусков в days_employed:', df['days_employed'].isnull().sum())

Кол-во пропусков в days_employed: 0


#### Ежемесячный доход

Доля пропущенных значений в 'total_income' существенна – 10%.  
Заменять данные о доходе на условный ноль нельзя, потому что это может исказить зависимость между уровнем дохода и возвратом кредита в срок. 

Правильнее будет заменить пустые значения медианными исходя их типа занятости и уровня образования (education_id и income_type), предполагая, что доход потенциально может зависеть от этих параметров.

Воспользуемся комбинацией groupby() и transform(), чтобы заполнить отсутствующие значения медианой.

In [6]:
df['total_income'] = df['total_income'].fillna(df.groupby(['education_id', 'income_type'])['total_income'].transform('median'))
print('Кол-во пропусков в total_income:', df['total_income'].isnull().sum())

Кол-во пропусков в total_income: 0


#### Вывод

На этапе предобработки данных были обнаружены синхронно отсутствующие значения в столбцах total_income и days_employed.

Отсутствие данных может быть вызвано:
- либо ошибками выгрузки данных из базы, 
- либо более вероятно, что кредит был выдан по упрощенной процедуре без необходимости предоставления данных о доходе и трудовом стаже.

Отсутствующие значения в трудовом стаже были заменены на условное значение -1, отличное от нуля, чтобы не искажать статистику данных по заемщикам с реальным нулевым стажем. Поскольку вопросы заказчика не предполагают выявление зависимости между продолжительностью трудового стажа и возвратом кредита в срок, то было принято решение не обрабатывать данные о стаже на предмет отрицательных значений и положительных значений большой разрядности.

Отсутствующие значения в уровне дохода были заменены на медиану уровня дохода исходя из типа занятости и уровня образования. Использование медианы обосновано тем, что средняя может быть искажена, если есть аномальные значения дохода внутри группы.

<a id="duplicates"></a> 
### Дубликаты

#### Дубликаты, связанные с использованием разных регистров написания

Чтобы исключить дубликаты, связанные с использованием разных регистров написания, приведем значения столбцов:
- 'education',
- 'family_status',
- 'income_type',
- 'gender'   

к нижнему регистру, воспользовавшись методом str.lower(). И после обработки вызовем списки уникальных значений в этих столбцах.

In [7]:
object_columns = ['education', 'family_status', 'income_type', 'gender']

print('Список уникальных значений')
print('--------------')

for column in object_columns:
    df[column] = df[column].str.lower()
    print(column, df[column].unique())
    print(' ')

Список уникальных значений
--------------
education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
 
family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
 
income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
 
gender ['f' 'm' 'xna']
 


Дубликаты, вызванные использованием разных регистров написания, отсуствуют.

#### Скрытые дубликаты

Проверим, нет ли скрытых дубликатов такого типа: одно и то же значение в 'education_id' соответствует разным значениям в столбце 'education'. Аналогичную проверку на скрытые дубликаты сделаем для пары столбцов 'family_status_id' – 'family_status'. 

Для этого отдельно сгруппируем данные по 'education_id' и 'family_status_id' и для каждого id найдем список уникальных значений в столбцах 'education' и 'family_status' соответственно.

In [8]:
df.groupby('education_id').agg({'education': 'unique'})

,education
education_id,
0,[высшее]
1,[среднее]
2,[неоконченное высшее]
3,[начальное]
4,[ученая степень]


In [9]:
df.groupby('family_status_id').agg({'family_status': 'unique'})

,family_status
family_status_id,
0,[женат / замужем]
1,[гражданский брак]
2,[вдовец / вдова]
3,[в разводе]
4,[не женат / не замужем]


Скрытых дубликатов нет.

#### Вывод

На этапе визуального анализа данных в столбце 'education' были обнаружены значения, написанные с использованием разных регистов. Можно предположить, что причиной их возникновения является тот факт, что поле 'education' заполняется сотрудниками банка вручную, а не путем выбора необходимого параметра из ограниченного списка.

Все значения в столбце 'education' приведены к нижнему регистру.

Аналогичная процедура также применена еще к 3-м столбцам типа object – 'family_status', 'income_type', 'gender', – чтобы исключить вероятность дублирования значений с разным написанием.

Проверка на скрытые дубликаты в 2-х парах столбцов  'education_id' – 'education' и 'family_status_id' – 'family_status' показала отсутствие таковых.

На основе выявленных ошибок можно рекомендовать банку доработку к форме ввода данных, чтобы поля типа object (текстовые поля) по возможности заполнялись путем выбора необходимого варианта из ограниченного списка, а не путем ручного ввода данных. 

<a id="mistakes"></a> 
### Прочие ошибки данных

Выведем максимальное и минимальное значения в столбцах типа int, чтобы исключить аномалии в данных:
- children, 
- dob_years,
- debt.

In [10]:
int_columns = ['children', 'dob_years', 'debt']
for column in int_columns:
    print(column)
    print('max:', df[column].max())
    print('min:', df[column].min())
    print(' ')

children
max: 20
min: -1
 
dob_years
max: 75
min: 0
 
debt
max: 1
min: 0
 


- В столбце 'children' есть явные ошибки данных – -1 ребенок. 20 детей скорее всего тоже является ошибкой.
- В столбце 'dob_years' встречается значение 0, что является ошибкой, поскольку кредитование начинается с 18 лет.

Возможно, это ошибки выгрузки или ошибка ручного ввода данных на этапе заполнения анкеты клиента.

- В солбце 'debt' с данными все нормально.

Далее обработаем найденные ошибки.

#### Количество детей

Посмотрим на распределение заемщиков по группам в зависимости от кол-ва детей.

In [11]:
childrens = df.groupby('children').agg(clients_qty = ('children', 'count')).reset_index()
childrens['clients_ratio, %'] = round(childrens['clients_qty'] / childrens['clients_qty'].sum() * 100, ndigits=1)
childrens

,children,clients_qty,"clients_ratio, %"
0,-1,47,0.2
1,0,14149,65.7
2,1,4818,22.4
3,2,2055,9.5
4,3,330,1.5
5,4,41,0.2
6,5,9,0.0
7,20,76,0.4


Доля клиентов с ошибочными данными в количестве детей (-1 и 20 детей) меньше 1%.  
Можно предположить, что -1 – на самом деле 1, а 20 – на самом деле 2. Поэтому заменим строки с -1 на 1, а 20 на 2. Статистику по заемщикам с 1-м и 2-мя детьми мы не исказим, потому что доля добавленных случаев 0.2% и 0.4% от числа заемщиков с 1-м и 2-мя детьми соответственно.

In [12]:
df['children'] = df['children'].replace(-1, 1) 
df['children'] = df['children'].replace(20, 2) 
df.groupby('children').agg(clients_qty = ('children', 'count'))

,clients_qty
children,
0,14149
1,4865
2,2131
3,330
4,41
5,9


#### Возраст клиента

Посмотрим, какие еще значения возраста менее 18 лет присутствует в данных и сколько строк имеют возраст меньше 18 лет.

In [13]:
print('Перечень возрастов до 18 лет:', df.query('dob_years < 18')['dob_years'].unique())
print('Кол-во строк с возрастом до 18 лет:', len(df.query('dob_years < 18')))
print('% строк с возрастом до 18 лет:', round(len(df.query('dob_years < 18')) / len(df) * 100, ndigits=1))

Перечень возрастов до 18 лет: [0]
Кол-во строк с возрастом до 18 лет: 101
% строк с возрастом до 18 лет: 0.5


0 лет – это единственное значение возраста, которое меньше 18 лет. В данных 101 строка с указанным возрастом 0 лет – это менее 1% данных. Причина данной ошибки может быть либо ошибка выгрузки данных, либо ошибка ручного ввода на этапе заполнения анкеты клиента.

Оставим эту ошибку без обработки, поскольку она не влияет на выводы, которые необходимо сделать по результатам анализа данных.

#### Вывод

Дальнейший визуальный анализ данных показал наличие следующих ошибок в данных:   

- -1 и 20 в значениях кол-ва детей (столбец 'children')    
Ошибочные значения были исправлены на реально возможные – 1 и 2 соответственно.


- 0 в значении возраста клиента (столбец 'dob_years')    
Ошибка оставлена без обработки, поскольку она не влияет на выводы, которые необходимо сделать по результатам анализа данных.

Вероятно, причиной данных ошибок является ручной ввод данных на этапе заполнения анкеты клиента. Можно рекомендовать банку доработку к форме ввода данных, чтобы данные поля проходили определенную автоматическую проверку:
- 'children' – на отсутствие отрицательных значений,
- 'dob_years' – на полное отсутствие значения или на значение меньше 18.

<a id="full_duplicates"></a> 
### Полные дубликаты

После проведенной на предыдущих шагах предобработки данных необходимо проверить данные на наличие полных дубликатов – строк с абсолютно идентичными данными во всех столбцах.

In [14]:
print('Кол-во полных дубликатов строк:', len(df[df.duplicated()]))
print('% полных дубликатов строк:', round(len(df[df.duplicated()]) / len(df) * 100, ndigits=1))

Кол-во полных дубликатов строк: 71
% полных дубликатов строк: 0.3


Процент полных дубликатов строк менее 1%. Удалим их из данных методом drop_duplicates().

In [15]:
df = df.drop_duplicates()
print('Кол-во строк после удаления полных дубликатов:', len(df))

Кол-во строк после удаления полных дубликатов: 21454


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


#### Вывод

Предобработанные данные были проверены на наличие полных дубликатов – строк с абсолютно идентичным набором данных во всех столбцах.
Было обнаружено 71 строка, имеющие абсолютные дубли. Это менее 1% от всех данных. Полные дубли были удалены из таблицы.

Причиной возникновения полных дублей может быть как ошибка выгрузки данных, так и случайное совпадение в данных у разных клиентов. Случайное совпадение данных у разные клиентов проверить невозможно, поскольку отсутствуют данные по id заемщика.

На будущее можно рекомендовать предоставлять выгрузку с включенным столбцом 'id_заемщика'.

<a id="correlation"></a> 
## Исследовательский анализ данных

Напишем пользовательскую функцию, которая на выходе будет получать таблицу с данными, сгруппированными по столбцу, указанному в аргументе функции, и рассчитанными для каждой группы значениями:
- общее кол-во клиентов – clients_qty,
- кол-во клиентов с задолженностью – debts_qty,
- % клиентов с задолженностью – debts_ratio.
- % клиентов в данной группе  – clients_ratio.

In [17]:
def correlation(metrics):
    df_grouped = (df.groupby(metrics) # группировка данных
                    .agg(clients_qty=('debt', 'count'), # расчет кол-ва заемщиков в группе
                         debts_qty=('debt', 'sum'), #расчет кол-ва заемщиков с задолженностью
                         debts_ratio=('debt', 'mean') # расчет доли заемщиков с задолженностью
                        )
                    .sort_values(by=('debts_ratio'), ascending=False) # сортировка по убыванию доли заемщиков с задолженностью
                 )
    df_grouped['clients_ratio, %'] = round(df_grouped['clients_qty'] / df_grouped['clients_qty'].sum() * 100, ndigits=1) # расчет значений столбца 'clients_ratio'
    df_grouped['debts_ratio'] = round(df_grouped['debts_ratio'] * 100, ndigits=1) # перевод значений столбца 'debts_ratio' в проценты
    df_grouped = df_grouped.rename(columns={'debts_ratio': 'debts_ratio, %'}) # переименование столбца 'debts_ratio'
    return df_grouped # вывод созданной таблицы

<a id="family_status_cor"></a>
### Анализ зависимости между семейным положением и возвратом кредита в срок

Применим созданную функцию correlation(metrics) к столбцу 'family_status'.

In [18]:
correlation('family_status')

,clients_qty,debts_qty,"debts_ratio, %","clients_ratio, %"
family_status,,,,
не женат / не замужем,2810,274,9.8,13.1
гражданский брак,4151,388,9.3,19.3
женат / замужем,12339,931,7.5,57.5
в разводе,1195,85,7.1,5.6
вдовец / вдова,959,63,6.6,4.5


#### Вывод

Чаще всего обращаются за кредитами семейные люди: 57.5% всех заемщиков – это женатые/замужние люди. Реже всего – люди ранее состоявшие в браке (разведенные и овдовевшие).

При этом видна очевидная зависимость между семейным положением заемщика и его кредитной дисциплиной: чем дальше заемщик в своем брачном статусе от официального брака (холост <= гражданский брак <= официальный брак), тем менее надежным заемщикам он является, т.е. вероятность возврата кредита в срок снижается.

Люди ранее состоявшие в браке (разведенные и овдовевшие) проявляют более высокий уровень дисциплины в вопросах погашения кредитов, чем люди в браке. При этом самые дисциплинированные – это овдовевшие клиенты.

<a id="children_cor"></a>
### Анализ зависимости между количеством детей и возвратом кредита в срок

Применим созданную функцию correlation(metrics) к столбцу 'children'.

In [19]:
correlation('children')

,clients_qty,debts_qty,"debts_ratio, %","clients_ratio, %"
children,,,,
4,41,4,9.8,0.2
2,2128,202,9.5,9.9
1,4855,445,9.2,22.6
3,330,27,8.2,1.5
0,14091,1063,7.5,65.7
5,9,0,0.0,0.0


Из распределения заемщиков по кол-ву детей видно, что чаще всего за кредитами обращаются люди, не имеющие детей – 66% всех заемщиков. При этом у них самый низкий показатель наличия задолженности по возврату кредитов – 7.5%.
Далее с увеличением количества детей число заемщиков уменьшается с каждым шагом в разы, а показатель наличия задолженности растет.

Однако в данных присутствуют категории, представленные малым кол-вом данных – заемщикам с 5, 4, 3 детьми. По этой причине выводы по ним могут быть не объективными. Можно укрупнить категории клиентов:
- нет детей,
- 1 ребенок,
- 2 ребенка и более.   

Для этого напишем пользовательскую функцию children_category, которая на вход берет данные о кол-ве детей, и возвращает категорию клиента. К столбцу с полученными категориями применим функцию correlation(metrics).

In [20]:
def children_category(children):
    if children == 0:
        return 0
    if children == 1:
        return 1
    else: return '2 and more'

In [21]:
df['children_category'] = df['children'].apply(children_category)
correlation('children_category')

,clients_qty,debts_qty,"debts_ratio, %","clients_ratio, %"
children_category,,,,
2 and more,2508,233,9.3,11.7
1,4855,445,9.2,22.6
0,14091,1063,7.5,65.7


#### Вывод

На укрупненных данных четко видны следующие зависимости:
- с увеличением кол-ва детей кол-во обращений/или одобрений кредитов уменьшается:  66% кредитов приходится на заемщиков, у которых нет детей, 26% на заемщиков с 1 ребенком и 12% на заемщиков с 2 и более детьми;
- с увеличением кол-ва детей вероятность появления задолженности по возврату кредита повышается: самый низкий уровень наличия задолженности демонстрируют клиенты без детей – 7.5%, самый высокий 9.3% – клиенты с 2 и более детьми.

<a id="income_cor"></a>
###  Анализ зависимости между уровнем дохода и возвратом кредита в срок

Cоздадим столбец 'total_income_category' с категориями заемщиков по уровню дохода, взяв за основу категории, применяемые в России:
- 0-50000 – 'F (<50)';
- 50000-100000 – 'E (50-100)';
- 100001-150000 – 'D (100-150)';
- 150001-250000 – 'C (150-250)';
- 250001-500000 – 'B (250-500)';
- 500001 и выше – 'A (>500)'.

Для распределения заемщиков по категориям напишем пользовательскую функци total_income_category, которая на вход берет величину дохода, и возвращает категорию клиента.

In [22]:
def total_income_category(income):
    if income <= 50000:
        return 'F (<50)'
    if 50000 < income <= 100000:
        return 'E (50-100)'
    if 100000 < income <= 150000:
        return 'D (100-150)'
    if 150000 < income <= 250000:
        return 'C (150-250)'
    if 250000 < income <= 500000:
        return 'B (250-500)'
    else: return 'A (>500)'

Создадим отдельный столбец 'total_income_category' с категориями по уровню дохода. Для этого воспользуемся методом apply(), который возмет значения столбца 'total_income' и применит к ним функцию total_income_category из своего аргумента.

In [23]:
df['total_income_category'] = df['total_income'].apply(total_income_category)  

Применим созданную функцию correlation(metrics) к столбцу 'total_income_category'.

In [24]:
correlation('total_income_category').reset_index().sort_values(by='total_income_category', ascending=False)

,total_income_category,clients_qty,debts_qty,"debts_ratio, %","clients_ratio, %"
5,F (<50),372,23,6.2,1.7
1,E (50-100),4091,331,8.1,19.1
0,D (100-150),6971,616,8.8,32.5
2,C (150-250),7207,577,8.0,33.6
3,B (250-500),2591,180,6.9,12.1
4,A (>500),222,14,6.3,1.0


#### Вывод

Полученные данные позволяют увидеть зависимость между уровнем дохода и возвратом кредита в срок, но она не является прямолинейной. 

Люди с низкими доходами (менее 50 тыс.) ограничены в доступе к кредитным ресурсам – их доля в общем количестве заемщиков всего 1.7%. Но их финансовая дисциплина на самом высоком уровне из всех выделенных категорий: доля задолженностей по возврату кредитов в срок только 6.2%. Можно предположить, что к данной категории изначально относятся более внимательно во время принятия решения о выдаче кредита, что может объяснять высокий уровень их кредитной дисциплины.

Далее, когда доходы начинают превышать 50 тыс., возможности получения кредитных ресурсов расширяются и кол-во кредитов увеличивается, но одновременно увеличивается показатель задолженности по возврату кредита в срок. Можно предположить, что заемщики, имея возможность взять гораздо большие по сумме и сроку кредиты, не учитывают необходимость более тщательного планирования и распоряжения личными финансами, поэтому не справляются с дополнительной финансовой нагрузкой. Самые ненадежные заемщики имеют доход 100-150 тыс., доля заемщиков с задолженностью 8.8%, но при этом они одни из самых многочисленных – 32.5% выборки.

Когда доходы начинают превышать 150 тыс. потребность в кредитных ресурсах остается на высоком уровне, но при этом кредитная дисциплины заемщиков начинает улучшаться. 

Когда доходы начинают превышать 250 тыс., видим снижение потребности в кредитах и дальнейшее улучшение кредитной дисциплины.

<a id="purpose_cor"></a>
### Анализ влияния цели кредитования на возврат кредита в срок

Посмотрим, какие цели кредитования встречаются в данных.

In [25]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Весь список целей можно распределить на 4 категории:
- операции с автомобилем,
- операции с недвижимостью,
- проведение свадьбы,
- получение образования.

Напишем пользовательскую функцию purpose_category, которая проверяет цели кредитования на содержание в них ключевых слов, на основании которых каждую из целей можно отнести к одной из 4-х категорий, и возвращает категорию целей кредитования.

In [26]:
def purpose_category(row):
    purpose = row['purpose']
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    if 'образован' in purpose:
        return 'получение образования'    
    if 'свадьб' in purpose:
        return 'проведение свадьбы'

Создадим отдельный столбец 'purpose_category' с категориями целей кредитования, для этого воспользуемся методом apply().

In [27]:
df['purpose_category'] = df.apply(purpose_category, axis=1)  

Применим созданную функцию correlation(metrics) к столбцу 'purpose_category'.

In [28]:
correlation('purpose_category')

,clients_qty,debts_qty,"debts_ratio, %","clients_ratio, %"
purpose_category,,,,
операции с автомобилем,4306,403,9.4,20.1
получение образования,4013,370,9.2,18.7
проведение свадьбы,2324,186,8.0,10.8
операции с недвижимостью,10811,782,7.2,50.4


#### Вывод

Самая популярная категория кредитов 50% – это кредиты на проведение операций с недвижимостью – имеет самый низкий показатель наличия задолженности по возврату кредитов в срок. Можно предположить, что условия предоставления кредита во многих случаях включают передачу недвижимости в залог банку до полного погашения кредита, что мотивирует заемщиков производить выплаты в срок. 

Самая малочисленная категория 11% – это проведение свадеб – также имеет довольно низкий уровень показателя просроченной задолженности – 8%. Можно предположить, что суммы кредитования на эти цели не очень большие. К тому же молодожены очень часто получают денежные подарки во время свадьбы, которыми впоследствии гасят большую часть кредита.

Автомобили и образование находятся примерно на одном уровне по популярности среди заемщиков – около 20%, но при этом имеют самый высокий показатель просроченной задолженности – 9.2-9.4%. Вероятно в силу того, что проходят как потребительское беззалоговое кредитование, к которому заемщики чаще относятся менее ответственно, чем к залоговому, потому что последствия допущения просрочки не настолько серьезные. По кредитам на образование можно сделать дополнительное предположение, что клиенты их берущие не имеют стабильного дохода, что впоследствии становится причиной нарушения сроков погашения.

<a id="conclusion"></a> 
## Общий вывод и рекомендации

На этапе предобработки были выявлены следующие **ПРОБЛЕМЫ В ДАННЫХ:**
- **симметричное отсутствие данных об общем трудовом стаже и ежемесячном доходе у 10% выборки.**   
 *Рекомендуем* обратить внимание на случаи одобрения и выдачи кредитов без предоставления данных о трудовом стаже и ежемесячном доходе.


- **плохоинтепретируемые данные о трудовом стаже – отрицательные значения и положительные значения большой разрядности.** 
 *Рекомендуем* обратить внимание на форму ввода данных, которая используется при оформлении кредитной заявки, и методики расчета дней трудового стажа на основе введенных данных, чтобы понять причину проблем с данными.


- **дубликаты, вызванные написанием с использованием разных регистров.**    
 *Рекомендуем* проанализировать форму ввода данных на предмет возможности заполнения текстовых полей путем выбора необходимого варианта из ограниченного списка, а не путем ручного ввода данных.


- **ошибки ручного ввода в данных по кол-ву детей и возрасту клиента.**   
 *Рекомендуем* проанализировать форму ввода данных на предмет возможности внедрения автоматической проверки данных, в частности:  
кол-во детей – на отсутствие отрицательных значений,   
возраст клиента – на полное отсутствие значения или на значение меньше 18.


- **полные дубликаты строк.**  
 *Рекомендуем* дополнить выгрузку данными по id клиента для корректной оценки кол-во полных дубликатов.


В ходе анализа платежеспособности клиентов были выявлены следующие **ЗАВИСИМОСТИ:**
- **семейное подожеже:** чем дальше заемщик в своем брачном статусе от официального брака (холост <= гражданский брак <= официальный брак), тем менее надежным заемщикам он является, т.е. вероятность возврата кредита в срок снижается. Самые ненадежные заемщики – это холостые клиенты. Клиенты ранее состоявшие в браке, но вышедшие из него по причине развода или смерти супруга, демонстрируют самую высокую финансовую дисциплину, но они и реже всех обращаются за кредитными средствами. Клиенты, состоящие в официальном браке, – самая многочисленная категория с высоким уровнем финансовой дисциплины.


- **количество детей:** с увеличением кол-ва детей кол-во обращений/или одобрений кредитов уменьшается, а вероятность появления задолженности по возврату кредита повышается.


- **уровень дохода:** пока клиент имеет доход до 150 тыс., то с ростом дохода наблюдается рост потребности в кредитных ресурсах, но уровень кредитной дисциплины при этом падает. Самые ненадежные заемщики имеют доход 100-150 тыс. При доходе 150-250 тыс. рост потребности в кредитах сохраняется, но замедляется, а вот финансовая дисциплина начинает повышаться. При достижении дохода в 250 тыс. и выше, спрос на кредиты падает, а дисциплина заемщиков в части своевременного погашения кредитов продолжает улучшаться.


- **цели кредитования:** самая популярная категория кредитов 50% – это кредиты на проведение операций с недвижимостью – имеет самый низкий показатель наличия задолженности по возврату кредитов в срок. Кредиты на цели, связанные с автомобями и образованием – суммарно около 40% выборки – имеют самый высокий показатель задолженности.